## **Training the model**

In [12]:
import os
import sys 
import pandas as pd
import numpy as np
import cv2
from torchvision.io import read_image
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, random_split, DataLoader

from torchvision.transforms import ToTensor
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision 
from torchvision import transforms
import timm
import segmentation_models_pytorch as smp
import wandb
from tqdm import tqdm

from dataset import *

sys.path.append('/'.join(os.getcwd().split('\\')[:2]) + '/utils')
from log import *

from model import unet_model 
from mask_to_rgb import * 

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
train_images_path = []
for root, dirs, files in os.walk(TRAIN_IMAGES_DIR):
    for file in files:
        path = os.path.join(root,file)
        train_images_path.append(path)
        
len(train_images_path)

1000

In [4]:
train_masks_path = []
for root, dirs, files in os.walk(TRAIN_MASKS_DIR):
    for file in files:
        path = os.path.join(root,file)
        train_masks_path.append(path)
        
len(train_masks_path)

1000

In [5]:
dataset = NeoPolypDataset(
    img_dir=TRAIN_IMAGES_DIR, 
    label_dir=TRAIN_MASKS_DIR, 
    resize=(256, 256), 
    transform=None
    )

In [6]:
images_data = []
labels_data = []
for x,y in dataset:
    images_data.append(x)
    labels_data.append(y)

In [7]:
model = unet_model
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [8]:
train_size = int(TRAIN_RATIO * len(images_data))
val_size = len(images_data) - train_size
train_dataset = CustomDataset(images_data[:train_size], labels_data[:train_size], transform=train_transformation)
val_dataset = CustomDataset(images_data[train_size:], labels_data[train_size:], transform=val_transformation)

train_loader = DataLoader(train_dataset, batch_size=BATCHSIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCHSIZE, shuffle=True)

In [9]:
wandb.login(
    key = "25283834ecbe7bd282505b0721ea3adcd8e789d3",
)
wandb.init(
    project = "UNet for Colonoscopy Polyp Segmentation"
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: fisherman611 (fisherman611-hanoi-university-of-science-and-technology). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Administrator\_netrc


In [10]:
# Move the model to the appropriate device
model.to(device)

# Define the loss function and initialize the best validation loss
criterion = nn.CrossEntropyLoss()
best_val_loss = float('inf')

# Create a progress bar for tracking epochs
epoch_bar = tqdm(total=EPOCHS, desc='Training Progress')

for epoch in range(EPOCHS):
    model.train()
    total_train_loss = 0

    # Training loop
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device).squeeze(1).long()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accumulate training loss
        total_train_loss += loss.item()
    
    model.eval()
    total_val_loss = 0

    # Validation loop
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device).squeeze(1).long()

            # Forward pass
            outputs = model(images)

            # Compute validation loss
            total_val_loss += criterion(outputs.float(), labels.long()).item()

    # Calculate average losses
    avg_train_loss = total_train_loss / len(train_loader)
    avg_val_loss = total_val_loss / len(val_loader)

    # Log progress and update the progress bar
    print(f"Epoch [{epoch + 1}/{EPOCHS}], Validation Loss: {avg_val_loss:.10f}")
    epoch_bar.set_postfix({'Train Loss': avg_train_loss, 'Val Loss': avg_val_loss})

    # Save the model if validation loss improves
    if total_val_loss < best_val_loss:
        best_val_loss = total_val_loss
        checkpoint = {
            'epoch': epoch,
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'loss': total_val_loss,
        }
        torch.save(checkpoint, f'{new_cwd}/checkpoint/model.pth')
    
    # Log metrics to wandb
    wandb.log({'Train Loss': avg_train_loss, 'Val Loss': avg_val_loss})

    # Update the epoch progress bar
    epoch_bar.update(1)

# Close the progress bar
epoch_bar.close()


Training Progress:   0%|          | 0/50 [00:16<?, ?it/s, Train Loss=0.664, Val Loss=0.352]

Epoch [1/50], Validation Loss: 0.3519981274


Training Progress:   2%|▏         | 1/50 [00:33<14:03, 17.22s/it, Train Loss=0.263, Val Loss=0.193]

Epoch [2/50], Validation Loss: 0.1927215595


Training Progress:   4%|▍         | 2/50 [00:49<13:22, 16.72s/it, Train Loss=0.157, Val Loss=0.127]

...

Epoch [48/50], Validation Loss: 0.0429415341


Training Progress:  98%|█████████▊| 49/50 [13:23<00:16, 16.03s/it, Train Loss=0.00959, Val Loss=0.0403]

Epoch [49/50], Validation Loss: 0.0402817191


Training Progress: 100%|██████████| 50/50 [13:40<00:00, 16.40s/it, Train Loss=0.00991, Val Loss=0.0374]

Epoch [50/50], Validation Loss: 0.0374240418


In [28]:
checkpoint = torch.load(f'{new_cwd}/checkpoint/model.pth')
model.load_state_dict(checkpoint['model'])
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
model.to(device)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_7760\3760274449.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(f'{new_cwd}/checkpoint/model.

Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [29]:
model.eval()
for i in os.listdir(TEST_DIR):
    img_path = os.path.join(TEST_DIR, i)
    ori_img = cv2.imread(img_path)
    ori_img = cv2.cvtColor(ori_img, cv2.COLOR_BGR2RGB)
    ori_w = ori_img.shape[0]
    ori_h = ori_img.shape[1]
    img = cv2.resize(ori_img, (256, 256))
    transformed = val_transformation(image=img)
    input_img = transformed["image"]
    input_img = input_img.unsqueeze(0).to(device)
    with torch.no_grad():
        output_mask = model.forward(input_img).squeeze(0).cpu().numpy().transpose(1,2,0)
    mask = cv2.resize(output_mask, (ori_h, ori_w))
    mask = np.argmax(mask, axis=2)
    mask_rgb = mask_to_rgb(mask, COLOR_DICT)
    mask_rgb = cv2.cvtColor(mask_rgb, cv2.COLOR_RGB2BGR)
    cv2.imwrite("{}/prediction/{}".format(new_cwd, i), mask_rgb) 

In [ ]:
def rle_to_string(runs):
    return ' '.join(str(x) for x in runs)

def rle_encode_one_mask(mask):
    pixels = mask.flatten()
    pixels[pixels > 225] = 255
    pixels[pixels <= 225] = 0
    use_padding = False
    if pixels[0] or pixels[-1]:
        use_padding = True
        pixel_padded = np.zeros([len(pixels) + 2], dtype=pixels.dtype)
        pixel_padded[1:-1] = pixels
        pixels = pixel_padded
    rle = np.where(pixels[1:] != pixels[:-1])[0] + 2
    if use_padding:
        rle = rle - 1
    rle[1::2] = rle[1::2] - rle[:-1:2]
    
    return rle_to_string(rle)

def rle2mask(mask_rle, shape=(3,3)):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

def mask2string(dir):
    strings = []
    ids = []
    ws, hs = [[] for i in range(2)]
    for image_id in os.listdir(dir):
        id = image_id.split('.')[0]
        path = os.path.join(dir, image_id)
        print(path)
        img = cv2.imread(path)[:,:,::-1]
        h, w = img.shape[0], img.shape[1]
        for channel in range(2):
            ws.append(w)
            hs.append(h)
            ids.append(f'{id}_{channel}')
            string = rle_encode_one_mask(img[:,:,channel])
            strings.append(string)
    r = {
        'ids': ids,
        'strings': strings,
    }
    return r


MASK_DIR_PATH = f'{new_cwd}/prediction'
dir = MASK_DIR_PATH
res = mask2string(dir)
df = pd.DataFrame(columns=['Id', 'Expected'])
df['Id'] = res['ids']
df['Expected'] = res['strings']

df.to_csv(f'{new_cwd}submission.csv', index=False)

d:/UNet-for-Colonoscopy-Polyp-Segmentation/prediction\019410b1fcf0625f608b4ce97629ab55.jpeg
d:/UNet-for-Colonoscopy-Polyp-Segmentation/prediction\02fa602bb3c7abacdbd7e6afd56ea7bc.jpeg
d:/UNet-for-Colonoscopy-Polyp-Segmentation/prediction\0398846f67b5df7cdf3f33c3ca4d5060.jpeg
d:/UNet-for-Colonoscopy-Polyp-Segmentation/prediction\05734fbeedd0f9da760db74a29abdb04.jpeg
d:/UNet-for-Colonoscopy-Polyp-Segmentation/prediction\05b78a91391adc0bb223c4eaf3372eae.jpeg
d:/UNet-for-Colonoscopy-Polyp-Segmentation/prediction\0619ebebe9e9c9d00a4262b4fe4a5a95.jpeg
d:/UNet-for-Colonoscopy-Polyp-Segmentation/prediction\0626ab4ec3d46e602b296cc5cfd263f1.jpeg
d:/UNet-for-Colonoscopy-Polyp-Segmentation/prediction\0a0317371a966bf4b3466463a3c64db1.jpeg
d:/UNet-for-Colonoscopy-Polyp-Segmentation/prediction\0a5f3601ad4f13ccf1f4b331a412fc44.jpeg
d:/UNet-for-Colonoscopy-Polyp-Segmentation/prediction\0af3feff05dec1eb3a70b145a7d8d3b6.jpeg
d:/UNet-for-Colonoscopy-Polyp-Segmentation/prediction\0fca6a4248a41e8db8b4ed633b